In [1]:
!pip install tensorflow


  Using cached tensorflow-2.20.0-cp313-cp313-win_amd64.whl.metadata (4.6 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.9.23-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-6.32.1-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached wrapt-1.17.3-cp313-cp313-win_amd64.whl.metadata (6.5 kB)
  Using cac

In [28]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt # Importamos o Matplotlib para ver o gráfico

# --- 1. DADOS DE TREINO (Continua o mesmo) ---
frases_de_treino = [
    # (Cole aqui a sua lista de frases_de_treino equilibrada)
    ("oi", "saudacao"), ("boa noite", "saudacao"), ("olá", "saudacao"),
    ("bom dia", "saudacao"), ("boa tarde", "saudacao"), ("e aí", "saudacao"),
    ("tudo bem?", "saudacao"),
    ("quais os combos?", "ver_combos"), ("me fala dos combos", "ver_combos"),
    ("o que vem no combo família?", "ver_combos"), ("tem combo?", "ver_combos"),
    ("queria saber sobre os combos", "ver_combos"),
    ("qual o preço do sasami?", "ver_porcoes_especificas"),
    ("tem coxinha da asa?", "ver_porcoes_especificas"),
    ("quanto custa as tiras crocantes?", "ver_porcoes_especificas"),
    ("qual o valor do peito de frango de 1kg", "ver_porcoes_especificas"),
    ("tem o chicken crunch?", "ver_sanduiches"), ("o que vem no lanche?", "ver_sanduiches"),
    ("queria um sanduíche", "ver_sanduiches"), ("quais os sanduiches?", "ver_sanduiches"),
    ("a parmegiana serve quantas pessoas?", "perguntar_sobre_item"),
    ("a mini caixa vem com molho?", "perguntar_sobre_item"),
    ("o peito de frango vem com fritas?", "perguntar_sobre_item"),
    ("o combo família tem refri?", "perguntar_sobre_item"),
    ("estão abertos agora?", "horario_funcionamento"), ("qual o horario?", "horario_funcionamento"),
    ("até que horas vcs entregam?", "horario_funcionamento"), ("vcs fecham que horas?", "horario_funcionamento"),
    ("abrem hoje?", "horario_funcionamento"),
    ("obrigado", "despedida"), ("valeu", "despedida"), ("tchau", "despedida"),
    ("até mais", "despedida"), ("obg", "despedida"), ("vlw", "despedida"),
]


# --- 2. PREPARAÇÃO DOS DADOS (Continua o mesmo) ---
frases = [item[0] for item in frases_de_treino]
intencoes = [item[1] for item in frases_de_treino]
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(frases)
sequences = tokenizer.texts_to_sequences(frases)
padded_sequences = pad_sequences(sequences, padding='post')
unique_labels = sorted(list(set(intencoes)))
label_map = {label: i for i, label in enumerate(unique_labels)}
labels = np.array([label_map[label] for label in intencoes])

# --- 3. O CÉREBRO V4 (COM AJUSTES FINOS) ---
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 32
max_length = padded_sequences.shape[1]
num_classes = len(unique_labels)

model = tf.keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.2), # <<<--- MUDANÇA 1: Dropout mais leve
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(num_classes, activation='softmax')
])

# --- 4. COMPILAÇÃO E TREINO (COM AJUSTES FINOS) ---
# <<<--- MUDANÇA 2: Definimos o otimizador Adam com um ritmo de aprendizado menor
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print("Iniciando o treinamento do Cérebro V4...")
history = model.fit(padded_sequences, labels, epochs=200, verbose=0)
print("Treinamento V4 finalizado!")

# --- 5. SALVANDO O NOVO CÉREBRO ---
model.save('cerebro_chatbot.keras')
print("Novo cérebro V4 salvo com sucesso!")

# --- 6. [NOVO!] VISUALIZANDO O APRENDIZADO ---
plt.plot(history.history['accuracy'])
plt.title('Acurácia do Modelo durante o Treinamento')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.show()

# --- NOSSO PLAYGROUND DE TESTES ---
prever_intencao("queria saber dos combos")
prever_intencao("tem o lanche de frango?")
prever_intencao("e aí mano")
prever_intencao("vcs fecham que horas?")
prever_intencao("qual o telefone de vcs?")

ModuleNotFoundError: No module named 'matplotlib'

In [24]:
import pickle

# --- Salvando o "Cérebro" (O Modelo Keras) ---
# O Keras tem uma função própria para salvar o modelo inteiro:
# a arquitetura, os pesos aprendidos, tudo.
model.save('cerebro_chatbot.keras')


# --- Salvando os "Tradutores" (os Tokenizers) com Pickle ---

# Salvando o tokenizer das frases
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Salvando o nosso mapa de intenções
with open('label_map.pickle', 'wb') as handle:
    pickle.dump(label_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

print("Cérebro (modelo) e Tradutores (tokenizers) salvos com sucesso na pasta do projeto!")

Cérebro (modelo) e Tradutores (tokenizers) salvos com sucesso na pasta do projeto!
